In [ ]:
import tensorflow as tf
import tensorflow_model_optimization as tfmot
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from skimage import transform
from tensorflow_model_optimization.quantization.keras import quantize_annotate_layer, quantize_apply


# from tensorflow.keras import layers
# from tensorflow_model_optimization.quantization.keras import quantize_model, QuantizeConfig
# from tensorflow_model_optimization.quantization.keras.default_8bit import Default8BitQuantizeScheme

print(tf.__version__)

In [ ]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

In [ ]:
# print(x_train[0])
print(x_train.shape)
x_train = transform.resize(x_train, (x_train.shape[0], 16, 16, 1))
x_test = transform.resize(x_test, (x_test.shape[0], 16, 16, 1))
x_train *= 256
x_train -= 128
x_train = x_train.astype(np.int8)
x_test *= 256
x_test -= 128
x_test = x_train.astype(np.int8)
# print(x_train[0])

In [ ]:
plt.imshow(x_train[0][:, :, 0], cmap='gray')

In [ ]:
y_train = tf.one_hot(y_train.astype(np.int32), depth=10)
y_test = tf.one_hot(y_test.astype(np.int32), depth=10)
x_train_flat = x_train.reshape(x_train.shape[0], -1)  # Reshapes to (60000, 784)
x_test_flat = x_test.reshape(x_test.shape[0], -1)  # Reshapes to (10000, 784)

In [ ]:
model = tf.keras.models.Sequential([
    quantize_annotate_layer(tf.keras.layers.Dense(32, activation='relu', dtype=tf.float16)),
    quantize_annotate_layer(tf.keras.layers.Dense(32, activation='relu', dtype=tf.float16)),
    quantize_annotate_layer(tf.keras.layers.Dense(10, activation='softmax', dtype=tf.float16))
])

model.compile(optimizer=tf.keras.optimizers.legacy.RMSprop(epsilon=1e-08), loss='categorical_crossentropy', metrics=['acc'])

In [ ]:
batch_size = 64
num_classes = 10
epochs = 5
history = model.fit(x_train_flat, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    validation_split=0.1,
                    callbacks=[])

In [ ]:
quantized_model = quantize_apply(model)
quantized_model.compile(optimizer=tf.keras.optimizers.legacy.RMSprop(epsilon=1e-08), loss='categorical_crossentropy', metrics=['acc'])

In [ ]:
np.set_printoptions(threshold=np.inf)

with open("weights_quantized.txt", "w") as f:
    for layer in quantized_model.layers:
        try:
            weights, biases = layer.get_weights()
            f.write(f"Weights for layer {layer.name}: {weights}\n")
            f.write(f"Biases for layer {layer.name}: {biases}\n")
        except: 
            pass
        finally:
            print(layer.get_weights())